In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
customers = pd.read_csv("../input/ecommerce-customers/Ecommerce Customers.csv")

In [ ]:
customers.head()

E-handelsfirma med base i New York City, der sælger tøj online, men de har også rådgivningssessioner i stil og tøj. Kunder kommer ind i butikken, har sessioner / møder med en personlig stylist, så kan de gå hjem og bestille enten i en mobilapp eller et websted til det tøj, de ønsker.

Virksomheden forsøger at beslutte, om de vil fokusere deres indsats på deres mobilapp-oplevelse eller deres hjemmeside. De har ansat dig på kontrakt for at hjælpe dem med at finde ud af det! Lad os komme igang!

In [ ]:
customers.describe()

In [ ]:
customers.info()

## Undersøgende dataanalyse

** Lad os udforske dataene! **

I resten af øvelsen bruger vi kun de numeriske data i csv-filen.
___
** Brug seaborn til at oprette en fællesplot til at sammenligne kolonnen Tid på websted og Årligt beløb brugt. Er korrelationen fornuftig? **

In [ ]:
sns.set_palette("GnBu_d")
sns.set_style('whitegrid')

In [ ]:
sns.jointplot(x='Time on Website',y='Yearly Amount Spent',data=customers)

In [ ]:
sns.jointplot(x='Time on App',y='Yearly Amount Spent',data=customers)

** Vi kan gøre brug  af jointplot til at oprette en 2D-sekskantplot, der sammenligner tid på app og medlemskabslængde. **

In [ ]:
# Mere tid på hjemmesiden er lig med = flere penge brugt.

In [ ]:
sns.jointplot(x='Time on App',y='Length of Membership',kind='hex',data=customers)

** Lad os undersøge disse typer relationer på tværs af hele datasættet. Brug af Seaborns Pairplot

In [ ]:
sns.pairplot(customers)

In [ ]:
sns.heatmap(customers.corr(), cmap="Blues", annot=True)


Baseret på dette plot,  kan vi se at  de mest korrelerede funktioner er "Length of Membership " og "Yearly Amount Spent"

Derfor oprettes der en lineær modelplot (ved hjælp af seaborn's lmplot) af den årlige mængde brugt i forhold til medlemskabets længde for at se  forholdet

In [ ]:
sns.lmplot(x = 'Length of Membership', y = 'Yearly Amount Spent', data = customers)




## Træne Linear Regresssion modellen med
## Træning og testdata

Nu hvor vi har udforsket dataene lidt, lad os gå videre og opdele dataene i trænings- og testsæt.
** Indstil en variabel X svarende til kundernes numeriske egenskaber og en variabel y svarende til kolonnen "Årligt beløb brugt". **

In [ ]:
y = customers['Yearly Amount Spent']

In [ ]:
X = customers[['Avg. Session Length', 'Time on App','Time on Website', 'Length of Membership']]

** Brug model_selection.train_test_split fra sklearn til at opdele dataene i trænings- og testsæt. Indstil teststørrelse = 0,3 og tilfældig tilstand = 101 **

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)



## Træning af modellen

Nu er det tid til at træne vores model på vores træningsdata!

** Importer LineærRegression fra sklearn.linear_model **

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
## Implementere en instans af Linear Regression modellen og kald den LM.

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit(X_train,y_train)

In [ ]:
## Printe koefficienter ud af modellen

In [ ]:
# The coefficients
print('Coefficients: \n', lm.coef_)

## Forudsigelse af testdata
Nu hvor vi har tilpasset vores model, lad os evaluere dens ydeevne ved at forudsige testværdierne!

** Brug lm.predict () til at forudsige X_test-datasættet. **

In [ ]:
predictions = lm.predict( X_test)

** Opret et scatterplot af de virkelige testværdier versus de forudsagte værdier. **

In [ ]:
plt.scatter(y_test,predictions)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

## Evaluering af modellen

Lad os evaluere vores modelydelse ved at beregne den resterende sum af kvadrater og den forklarede variansscore (R ^ 2).

** Beregn den gennemsnitlige absolutte fejl, den gennemsnitlige kvadratfejl og den rodmidlet kvadrede fejl.  **

In [ ]:
from sklearn import metrics

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

## Residuals Plot 
Du skulle have fået en meget god model med en god pasform. Lad os hurtigt undersøge resterne for at sikre, at alt var okay med vores data.

** Plot et histogram over restprodukterne, og sørg for, at det ser normalt fordelt ud. Brug enten søfødt distplot eller bare plt.hist (). **

In [ ]:
sns.distplot((y_test-predictions),bins=50);

## Konklusion
Vi vil stadig finde ud af svaret på det oprindelige spørgsmål. Fokuserer vi vores effekt på mobilapp eller webstedsudvikling? Eller måske betyder det ikke engang rigtig noget, og medlemskabstid er det, der er virkelig vigtigt. Lad os se, om vi overhovedet kan fortolke koefficienterne for at få en idé.



In [ ]:
coeffecients = pd.DataFrame(lm.coef_,X.columns)
coeffecients.columns = ['Coeffecient']
coeffecients

Fortolkning af koefficienterne:

- Holder alle andre funktioner fast, en stigning på 1 enhed i ** Gns. Sessionslængde ** er forbundet med en ** stigning på 25,98 samlede brugte dollars **.
- Når alle andre funktioner er fastholdt, er en stigning på 1 enhed i ** Time on App ** forbundet med en ** stigning på 38,59 samlede brugte dollars **.
- Når alle andre funktioner er løst, er en stigning på 1 enhed i ** Tid på websted ** forbundet med en ** stigning på 0,19 samlede brugte dollars **.
- Når alle andre funktioner er fast, er en stigning på 1 enhed i ** Medlemskabets længde ** forbundet med en ** stigning på 61,27 samlede brugte dollars **.

 ## Spørgsmålet lyder så 
    ** Tror du, at virksomheden bør fokusere mere på deres mobilapp eller på deres hjemmeside? **

## Jeg ville som Data Scientist anbefale 2 måder

### Udvikl webstedet for at indhente ydeevnen for mobilappen eller udvikle appen mere, da det er det, der fungerer bedre. Denne form for svar afhænger virkelig af de andre faktorer, der foregår hos virksomheden. Du vil sandsynligvis gerne undersøge forholdet mellem medlemskabets længde og appen eller hjemmesiden, før du kommer til en konklusion!